In [3]:
from types import SimpleNamespace
from extraction import *

args = SimpleNamespace(
        input_dir="../data/",
        output_dir="../results/",
        tmp_dir="/scratch",
        diameter=0,
        anatomical_only=2,
        denoise=False,
        cellprob_threshold=0.0,
        flow_threshold=1.5,
        spatial_hp_cp=0,
        pretrained_model="cyto",
        use_suite2p_neuropil=False,
        contour_video=False,
    )

In [17]:
session, data_description, subject = get_metdata(input_dir)

In [19]:
    output_dir = Path(args.output_dir).resolve()
    input_dir = Path(args.input_dir).resolve()
    tmp_dir = Path(args.tmp_dir).resolve()
    session, data_description, subject = get_metdata(input_dir)
    subject_id = subject.get("subject_id", "")
    name = data_description.get("name", "")
    setup_logging("aind-ophys-extraction-suite2p", mouse_id=subject_id, session_name=name)
    if next(input_dir.rglob("*decrosstalk.h5"), ""):
        input_fn = next(input_dir.rglob("*decrosstalk.h5"))
    else:
        input_fn = next(input_dir.rglob("*registered.h5"))
    parent_directory = input_fn.parent
    if session is not None and "Bergamo" in session["rig_id"]:
        motion_corrected_fn = bergamo_segmentation(input_fn, session, temp_dir=tmp_dir)
    else:
        motion_corrected_fn = input_fn
    if not data_description or "multiplane" in data_description.get("name", ""):
        unique_id = motion_corrected_fn.parent.parent.name
    else:
        unique_id = "_".join(str(data_description["name"]).split("_")[-3:])

    frame_rate = get_frame_rate(session)

    output_dir = make_output_directory(output_dir, unique_id)

AWS credentials not found.
Could not connect to boto3. Logging to console only
Starting


In [24]:
    # Run Cellpose via Suite2p to get ROI seeds
    # =========================================
    # Set suite2p args.
    suite2p_args = suite2p.default_ops()
    # Overwrite the parameters for suite2p that are exposed
    suite2p_args["diameter"] = args.diameter
    suite2p_args["anatomical_only"] = args.anatomical_only
    suite2p_args["cellprob_threshold"] = args.cellprob_threshold
    suite2p_args["flow_threshold"] = args.flow_threshold
    suite2p_args["spatial_hp_cp"] = args.spatial_hp_cp
    suite2p_args["pretrained_model"] = args.pretrained_model
    suite2p_args["denoise"] = args.denoise
    suite2p_args["save_path0"] = args.tmp_dir
    # Here we overwrite the parameters for suite2p that will not change in our
    # processing pipeline. These are parameters that are not exposed to
    # minimize code length. Those are not set to default.
    suite2p_args["h5py"] = str(motion_corrected_fn)
    suite2p_args["data_path"] = []
    suite2p_args["roidetect"] = True
    suite2p_args["do_registration"] = 0
    suite2p_args["spikedetect"] = False
    suite2p_args["fs"] = frame_rate
    suite2p_args["neuropil_extract"] = True

    # determine nbinned from bin_duration and fs
    suite2p_args["bin_duration"] = 3.7  # The duration of time (in seconds) that
    # should be considered 1 bin for Suite2P ROI detection purposes. Requires
    # a valid value for 'fs' in order to derive an
    # nbinned Suite2P value. This allows consistent temporal downsampling
    # across movies with different lengths and/or frame rates.
    with h5py.File(suite2p_args["h5py"], "r") as f:
        nframes = f["data"].shape[0]
    bin_size = suite2p_args["bin_duration"] * suite2p_args["fs"]
    suite2p_args["nbinned"] = int(nframes / bin_size)
    try:
        suite2p.run_s2p(suite2p_args)
    except IndexError:  # raised when no ROIs found
        pass

    # load in the rois from the stat file and movie path for shape
    with h5py.File(str(motion_corrected_fn), "r") as open_vid:
        dims = open_vid["data"][0].shape
    if len(list(Path(args.tmp_dir).rglob("stat.npy"))):
        suite2p_stat_path = str(next(Path(args.tmp_dir).rglob("stat.npy")))
        suite2p_stats = np.load(suite2p_stat_path, allow_pickle=True)

{}
FOUND BINARIES AND OPS IN ['/scratch/suite2p/plane0/ops.npy']
removing previous detection and extraction files, if present
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not running registration, ops['do_registration']=0
binary path: /scratch/suite2p/plane0/data.bin
NOTE: Applying builtin classifier at /opt/conda/lib/python3.9/site-packages/suite2p/classifiers/classifier.npy
----------- ROI DETECTION
Binning movie in chunks of length 113
Binned movie of size [22,90,90] created in 0.03 sec.


>>>> using CPU
>> cyto << model set to be used


>>>> CELLPOSE finding masks in mean_img
!NOTE! diameter set to 0 or None, diameter will be estimated by cellpose


/opt/conda/lib/python3.9/site-packages/cellpose/resnet_torch.py:207: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(filename, map_location=torch.devic

>>>> 4 masks detected, median diameter = 10.20 
Detected 4 ROIs, 6.60 sec
After removing overlaps, 4 ROIs remain
----------- Total 8.47 sec.
----------- EXTRACTION
Masks created, 0.03 sec.


/opt/conda/lib/python3.9/site-packages/suite2p/extraction/extract.py:125: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  Fi[n] = np.dot(data[:, cell_ipix[n]], cell_lam[n])


Extracted fluorescence from 4 ROIs in 2500 frames, 6.56 sec.
----------- Total 6.62 sec.
----------- CLASSIFICATION
['skew', 'npix_norm', 'compact']
Plane 0 processed in 15.44 sec (can open in GUI).
total = 15.49 sec.
TOTAL RUNTIME 15.49 sec


In [75]:
from scipy.sparse import coo_matrix, csc_matrix, hstack

In [28]:
def csc_append(a, b):
    """ Takes in 2 csc_matrices and appends the second one to the right of the first one.
    Much faster than scipy.sparse.hstack but assumes the type to be csc and overwrites
    the first matrix instead of copying it. The data, indices, and indptr still get copied."""
    a.data = np.concatenate((a.data, b.data))
    a.indices = np.concatenate((a.indices, b.indices))
    a.indptr = np.concatenate((a.indptr, (b.indptr + a.nnz)[1:]))
    a._shape = (a.shape[0], a.shape[1] + b.shape[1])

In [76]:
%%timeit
rois = [
    coo_matrix((roi["lam"], (roi["ypix"], roi["xpix"])), shape=dims)
    for roi in suite2p_stats
]
Ain = csc_matrix((np.prod(dims), 0), dtype="f4")    
for roi in rois:
    csc_append(Ain, csc_matrix(roi.reshape((-1,1), order="F"))) 

1.04 ms ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [77]:
%%timeit
Ain2 = hstack([
                coo_matrix((roi["lam"], (roi["ypix"], roi["xpix"])), shape=dims).reshape((-1,1), order="F")
                for roi in suite2p_stats
            ]).tocsc()

1.01 ms ± 4.57 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [78]:
%%timeit
Ain3 = hstack([
                coo_matrix((roi["lam"], (roi["ypix"], roi["xpix"])), shape=dims).reshape((-1,1), order="F").tocsc()
                for roi in suite2p_stats
            ])

999 µs ± 3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [79]:
Ain != Ain3

<8100x4 sparse matrix of type '<class 'numpy.bool'>'
	with 0 stored elements in Compressed Sparse Column format>

In [80]:
Ain != Ain2

<8100x4 sparse matrix of type '<class 'numpy.bool'>'
	with 0 stored elements in Compressed Sparse Column format>